In [1]:
# This extension allows us to sync changes from utils without restarting the Jupyter kernel.
%reload_ext autoreload
%autoreload 1
%aimport src.utils

import torch
import torch.nn.functional as F
from src import utils
from tqdm import tqdm

ModuleNotFoundError: No module named 'sparsecoding'

In [3]:
img_path = "path/to/IMAGES.mat"
imgs = utils.load_data(img_path)
utils.show_imgs(imgs)

NameError: name 'utils' is not defined

In [4]:
# Sparse nonlinearity for the sparsity penalty; we use |x|, but there are other possibilities.
def C(x): return torch.abs(x)
def C_prime(x): return torch.sign(x)

# Calculate two terms of energy function according to equation 2.
def E_terms(I, Ihat, a, lambd):
    mse = F.mse_loss(Ihat, I).data
    sparseness = lambd * torch.sum(C(a))
    return mse, sparseness 

In [5]:
# Tunable parameters.
epochs = 3000
num_infer_iters = 300
batch_size = 100
lambd = 0.8
phi_lr = 1e-3
eta = 1e-3

# Initialize image variables.
num_bases = 64  # Size of feature dictionary.
patch_size = 8  # Each image is 8x8 pixels.
N = patch_size**2  # Number of pixels per image.

# Setting seed may be helpful for debugging and reproducibility.
seed = 0
rng = torch.Generator()
torch.manual_seed(seed)

# Initialize random dictionary.
phi = torch.randn(N, num_bases)
phi = F.normalize(phi, dim=0)

NameError: name 'torch' is not defined

In [7]:
patches = utils.create_patches(imgs, epochs, batch_size, N, rng)
utils.show_patches(patches[0,:, :], title="0th batch")

NameError: name 'utils' is not defined

In [8]:
# Store losses and phi snapshot for plotting later.
phi_snapshot_steps = 500
mses = []
sparse_costs = []

for i in tqdm(range(epochs)):
    I = patches[i, :, :]
    a = torch.zeros(batch_size, num_bases)
    Ihat = torch.mm(phi, a.T).T
    r = I - Ihat

    # This inner for loop uses Euler's method to infer coefficients a.
    for _ in range(num_infer_iters):
        da = eta * (torch.mm(r, phi) - (lambd * C_prime(a)))  # Equation 3.
        a += da

    # Update dictionary based on gradient with respect to phi.
    Ihat = torch.mm(phi, a.T).T 
    dphi = phi_lr * (torch.mm((I - Ihat).T, a))  # Equation 4.
    phi += dphi

    # Normalize each basis to unit length.
    phi = F.normalize(phi, dim=0)
    # Get phi snapshot to show later.
    if i == phi_snapshot_steps:  
        phi_snapshot = phi

    # Calculate energy for plotting purposes.
    mse, sparse_cost = E_terms(I, Ihat, a, lambd)
    mses.append(mse)
    sparse_costs.append(sparse_cost)

NameError: name 'tqdm' is not defined

In [9]:
utils.show_dict(phi, "dictionary")

NameError: name 'utils' is not defined

In [10]:
utils.show_patches_sbs(I, Ihat)

NameError: name 'utils' is not defined

In [11]:
utils.plot_loss_sbs(mses[:phi_snapshot_steps], sparse_costs[:phi_snapshot_steps])
utils.show_dict(phi_snapshot, "dictionary at {} epochs".format(phi_snapshot_steps))

NameError: name 'utils' is not defined

In [12]:
# Plot losses.
utils.plot_loss_sbs(mses, sparse_costs)
utils.show_dict(phi, "dictionary at {} epochs".format(epochs))

NameError: name 'utils' is not defined

In [13]:
img_id = 0
coeff = a[img_id]
utils.show_components(phi, coeff, dpi=150)

NameError: name 'a' is not defined

In [14]:
utils.coeff_grid(a)

NameError: name 'utils' is not defined